In [ ]:
# Run this cell first. It installs required libraries.
!pip install -q pandas scikit-learn xgboost joblib matplotlib seaborn lightgbm \
    snowflake-connector-python "snowflake-connector-python[pandas]" \
    category-encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 req

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datetime import datetime
print("Imports done.")


Imports done.


In [ ]:
# Upload residential_gold.csv to /content beforehand
PATH = "/content/RESIDENTIAL_GOLD.csv"  # change if needed
assert os.path.exists(PATH), f"File not found: {PATH} — upload your RESIDENTIAL_GOLD.csv to Colab /content"
df = pd.read_csv(PATH)
print("Loaded:", PATH)
df.shape


Loaded: /content/RESIDENTIAL_GOLD.csv


(14528, 15)

In [ ]:
# Inspect top rows and columns
display(df.head(3))
print("Columns:", df.columns.tolist())


,PROPERTY_ID,NAME,PROPERTY_TITLE,PRICE,TOTAL_AREA,PRICE_PER_SQFT,BATHS,BALCONY,LOCATION,CITY,LOCALITY,DESCRIPTION_LENGTH,PRICE_CATEGORY,BATH_CATEGORY,DESCRIPTION
0,0,Casagrand ECR 14,"4 BHK Flat for sale in Kanathur Reddikuppam, C...",19900000,2583,7704.219899,4,True,"Kanathur Reddikuppam, Chennai",Chennai,Kanathur Reddikuppam,603,Luxury,Premium,Best 4 BHK Apartment for modern-day lifestyle ...
1,1,"Ramanathan Nagar, Pozhichalur,Chennai",10 BHK Independent House for sale in Pozhichal...,22500000,7000,3214.285714,6,True,"Ramanathan Nagar, Pozhichalur,Chennai",Chennai,Ramanathan Nagar,603,Luxury,Premium,Looking for a 10 BHK Independent House for sal...
2,2,DAC Prapthi,"3 BHK Flat for sale in West Tambaram, Chennai",10000000,1320,7575.757576,3,False,"Kasthuribai Nagar, West Tambaram,Chennai",Chennai,Kasthuribai Nagar,603,Mid Range,Premium,"Property for sale in Tambaram, Chennai. This 3..."


Columns: ['PROPERTY_ID', 'NAME', 'PROPERTY_TITLE', 'PRICE', 'TOTAL_AREA', 'PRICE_PER_SQFT', 'BATHS', 'BALCONY', 'LOCATION', 'CITY', 'LOCALITY', 'DESCRIPTION_LENGTH', 'PRICE_CATEGORY', 'BATH_CATEGORY', 'DESCRIPTION']


In [ ]:
# Convert numeric columns safely and create derived features
df = df.copy()

# Normalize column names (keep uppercase from CSV, just strip spaces)
df.columns = [c.strip() for c in df.columns]

# Ensure numeric types (uppercase column names)
for col in ['PRICE','TOTAL_AREA','PRICE_PER_SQFT']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Baths -> integer-ish
if 'BATHS' in df.columns:
    df['BATHS'] = pd.to_numeric(df['BATHS'], errors='coerce').fillna(0).astype(int)
else:
    df['BATHS'] = 0

# Balcony -> binary flag
if 'BALCONY' in df.columns:
    df['BALCONY_FLAG'] = df['BALCONY'].astype(str).str.strip().str.lower().map({'yes':1,'true':1,'1':1}).fillna(0).astype(int)
else:
    df['BALCONY_FLAG'] = 0

# Description length
if 'DESCRIPTION' in df.columns:
    df['DESCRIPTION_LENGTH'] = df['DESCRIPTION'].astype(str).apply(len)
else:
    df['DESCRIPTION_LENGTH'] = 0

# If PROPERTY_ID not present, create surrogate from row index
if 'PROPERTY_ID' not in df.columns:
    df['PROPERTY_ID'] = np.arange(1, len(df)+1)

# Drop rows with missing PRICE or TOTAL_AREA
initial_rows = len(df)
df = df.dropna(subset=['PRICE','TOTAL_AREA'])
print(f"Dropped {initial_rows - len(df)} rows with missing PRICE/TOTAL_AREA. Remaining rows: {len(df)}")

df.sample(3)


Dropped 0 rows with missing PRICE/TOTAL_AREA. Remaining rows: 14528


,PROPERTY_ID,NAME,PROPERTY_TITLE,PRICE,TOTAL_AREA,PRICE_PER_SQFT,BATHS,BALCONY,LOCATION,CITY,LOCALITY,DESCRIPTION_LENGTH,PRICE_CATEGORY,BATH_CATEGORY,DESCRIPTION,baths,balcony_flag,description_length,property_id,BALCONY_FLAG
10583,10583,"malwadi ,Talegaon Dabhade, Pune",2 BHK Independent House for sale in Talegaon D...,6000000,1053,5698.005698,3,False,"Malwadi ,Talegaon Dabhade, Pune",Pune,Malwadi,603,Mid Range,Premium,2 BHK Independent House for sale in Talegaon D...,0,0,0,10584,0
12604,12604,Classic Apartment,"4 BHK Flat for sale in Sector 12 Dwarka, New D...",25000000,2150,11627.906980,4,True,"Sector 12 Dwarka, New Delhi",New Delhi,Sector 12 Dwarka,603,Luxury,Premium,4 BHK Apartment for sale in Delhi. This proper...,0,0,0,12605,1
8811,8811,Unimark Springfield,"2 BHK Flat for sale in Rajarhat, Kolkata",6000000,1300,4615.384615,3,True,"Gopalpur Gram, Rajarhat,Kolkata",Kolkata,Gopalpur Gram,603,Mid Range,Premium,One of the finest property in Gopalpur Gram is...,0,0,0,8812,1


In [ ]:
FEATURES = []

# Select only valid features based on your CSV
for c in ['TOTAL_AREA','PRICE_PER_SQFT','BATHS','BALCONY_FLAG','DESCRIPTION_LENGTH','CITY','PRICE_CATEGORY']:
    if c in df.columns:
        FEATURES.append(c)

TARGET = 'PRICE'   # <-- UPPERCASE FIX (IMPORTANT)

print("Using features:", FEATURES)
print("Target:", TARGET)



Using features: ['TOTAL_AREA', 'PRICE_PER_SQFT', 'BATHS', 'BALCONY_FLAG', 'DESCRIPTION_LENGTH', 'CITY', 'PRICE_CATEGORY']
Target: PRICE


In [ ]:
X = df[FEATURES].copy()
y = df[TARGET].copy()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (11622, 7) Test: (2906, 7)


In [ ]:
# Numeric vs categorical lists
num_feats = [c for c in FEATURES if c in ['TOTAL_AREA','PRICE_PER_SQFT','BATHS','DESCRIPTION_LENGTH','BALCONY_FLAG']]
cat_feats = [c for c in FEATURES if c not in num_feats]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='UNKNOWN')),

    # FIXED: sklearn >= 1.2 uses sparse_output instead of sparse
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_feats),
    ('cat', cat_transformer, cat_feats)
], remainder='drop')

print("Numeric:", num_feats)
print("Categorical:", cat_feats)



Numeric: ['TOTAL_AREA', 'PRICE_PER_SQFT', 'BATHS', 'BALCONY_FLAG', 'DESCRIPTION_LENGTH']
Categorical: ['CITY', 'PRICE_CATEGORY']


In [ ]:
pipe_lr = Pipeline([('pre', preprocessor), ('model', LinearRegression())])

pipe_rf = Pipeline([
    ('pre', preprocessor),
    ('model', RandomForestRegressor(n_jobs=-1, random_state=42))
])

# FIXED XGB version
pipe_xgb = Pipeline([
    ('pre', preprocessor),
    ('model', xgb.XGBRegressor(
        n_estimators=200,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.9,
        colsample_bytree=0.9,
        n_jobs=-1,
        random_state=42
        # removed objective to avoid version conflict
    ))
])

print("Pipelines defined (fixed XGB).")



Pipelines defined (fixed XGB).


In [ ]:
def evaluate_and_print(model, X_train, X_test, y_train, y_test, name="model"):
    model.fit(X_train, y_train)

    pred_train = model.predict(X_train)
    pred_test  = model.predict(X_test)

    # FIX: compute RMSE manually (no "squared" argument)
    rmse_train = mean_squared_error(y_train, pred_train) ** 0.5
    rmse_test  = mean_squared_error(y_test, pred_test) ** 0.5

    mae_train = mean_absolute_error(y_train, pred_train)
    mae_test  = mean_absolute_error(y_test, pred_test)

    r2_train = r2_score(y_train, pred_train)
    r2_test  = r2_score(y_test, pred_test)

    print(f"=== {name} TRAIN ===")
    print(f"RMSE: {rmse_train:.2f} | MAE: {mae_train:.2f} | R2: {r2_train:.3f}")
    print()
    print(f"=== {name} TEST ===")
    print(f"RMSE: {rmse_test:.2f} | MAE: {mae_test:.2f} | R2: {r2_test:.3f}")
    print("-"*50)

    return model, pred_test




In [ ]:
pipe_lr, pr_lr = evaluate_and_print(pipe_lr, X_train, X_test, y_train, y_test, "LinearRegression")
pipe_rf, pr_rf = evaluate_and_print(pipe_rf, X_train, X_test, y_train, y_test, "RandomForest")
pipe_xgb, pr_xgb = evaluate_and_print(pipe_xgb, X_train, X_test, y_train, y_test, "XGBoost")



=== LinearRegression TRAIN ===
RMSE: 10270593.54 | MAE: 3152067.70 | R2: 0.727

=== LinearRegression TEST ===
RMSE: 7409508.86 | MAE: 2878280.27 | R2: 0.720
--------------------------------------------------
=== RandomForest TRAIN ===
RMSE: 2180189.52 | MAE: 138403.42 | R2: 0.988

=== RandomForest TEST ===
RMSE: 1843542.63 | MAE: 203791.07 | R2: 0.983
--------------------------------------------------
=== XGBoost TRAIN ===
RMSE: 509648.91 | MAE: 252721.47 | R2: 0.999

=== XGBoost TEST ===
RMSE: 2151538.53 | MAE: 456295.78 | R2: 0.976
--------------------------------------------------


In [ ]:
# Cell 13 (fixed) — Quick model comparison table (safe RMSE computation)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def metrics_safe(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred) ** 0.5   # manual RMSE (no squared arg)
    mae  = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    return {'RMSE': rmse, 'MAE': mae, 'R2': r2}

rows = []
candidates = [
    ('Linear', pipe_lr, 'pr_lr'),
    ('RF',     pipe_rf, 'pr_rf'),
    ('XGB',    pipe_xgb, 'pr_xgb')
]

for name, model_obj, preds_name in candidates:
    # If preds variable exists (from previous evaluate), use it; else predict now
    preds = globals().get(preds_name, None)
    if preds is None:
        try:
            preds = model_obj.predict(X_test)
        except Exception as e:
            print(f"Warning: could not get predictions for {name}: {e}")
            continue
    m = metrics_safe(y_test, preds)
    rows.append({'model': name, 'RMSE': m['RMSE'], 'MAE': m['MAE'], 'R2': m['R2']})

cmp_df = pd.DataFrame(rows).sort_values('RMSE').reset_index(drop=True)
cmp_df


,model,RMSE,MAE,R2
0,RF,1.843543e+06,2.037911e+05,0.982662
1,XGB,2.151539e+06,4.562958e+05,0.976385
2,Linear,7.409509e+06,2.878280e+06,0.719926


In [ ]:
# Cell 14 — SAFE RandomForest tuning (works on all sklearn versions)

from sklearn.model_selection import RandomizedSearchCV

# Parameter search space
param_dist = {
    'model__n_estimators': [100, 200, 350],
    'model__max_depth': [None, 10, 15, 20],
    'model__min_samples_split': [2, 5, 10],
    'model__min_samples_leaf': [1, 2, 4]
}

print("Starting RandomForest tuning... (may take 20–40 seconds)")

rs = RandomizedSearchCV(
    pipe_rf,
    param_distributions=param_dist,
    n_iter=6,                 # number of random combinations to try
    cv=3,                     # 3-fold cross validation
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

rs.fit(X_train, y_train)

print("\nBest parameters found:")
print(rs.best_params_)

best_rf = rs.best_estimator_

# Evaluate tuned model
pred_test_rf = best_rf.predict(X_test)

rmse = mean_squared_error(y_test, pred_test_rf) ** 0.5
mae  = mean_absolute_error(y_test, pred_test_rf)
r2   = r2_score(y_test, pred_test_rf)

print("\n=== Tuned RandomForest Performance ===")
print(f"RMSE: {rmse:.2f}")
print(f"MAE : {mae:.2f}")
print(f"R²  : {r2:.3f}")
print("--------------------------------------")


Starting RandomForest tuning... (may take 20–40 seconds)
Fitting 3 folds for each of 6 candidates, totalling 18 fits

Best parameters found:
{'model__n_estimators': 350, 'model__min_samples_split': 2, 'model__min_samples_leaf': 1, 'model__max_depth': 20}

=== Tuned RandomForest Performance ===
RMSE: 1939793.22
MAE : 207350.63
R²  : 0.981
--------------------------------------


In [ ]:
# Cell 15 — Auto-select best model + save it

import joblib

print("Selecting the best model based on RMSE...\n")

models_to_compare = []

# Add tuned RF if available
if 'best_rf' in globals():
    models_to_compare.append(('RF_TUNED', best_rf))

# Add original three
models_to_compare.append(('LINEAR', pipe_lr))
models_to_compare.append(('RF', pipe_rf))
models_to_compare.append(('XGB', pipe_xgb))

best_model_name = None
best_model_obj = None
best_rmse = float('inf')

results = []  # to print comparison at end

for name, model in models_to_compare:
    try:
        preds = model.predict(X_test)
        rmse = mean_squared_error(y_test, preds) ** 0.5  # manual RMSE
        results.append((name, rmse))
        print(f"{name} RMSE: {rmse:.4f}")

        if rmse < best_rmse:
            best_rmse = rmse
            best_model_name = name
            best_model_obj = model
    except Exception as e:
        print(f"Error evaluating {name}: {e}")

print("\n=======================================")
print(f"BEST MODEL SELECTED: {best_model_name}")
print(f"BEST RMSE: {best_rmse:.4f}")
print("=======================================\n")

# Save model
os.makedirs("ai", exist_ok=True)
joblib.dump(best_model_obj, "ai/model_best.pkl")

print("Saved best model to: ai/model_best.pkl")


Selecting the best model based on RMSE...

RF_TUNED RMSE: 1939793.2225
LINEAR RMSE: 7409508.8608
RF RMSE: 1843542.6252
XGB RMSE: 2151538.5329

BEST MODEL SELECTED: RF
BEST RMSE: 1843542.6252

Saved best model to: ai/model_best.pkl


In [ ]:
# Cell 15.5 — Load GOLD dataset for predictions

import pandas as pd

df_gold = pd.read_csv("/content/RESIDENTIAL_GOLD.csv")   # <-- change path if needed
print("Loaded GOLD dataset:")
df_gold.head()

# Define the exact feature columns used in the model
feature_cols = [
    'TOTAL_AREA',
    'PRICE_PER_SQFT',
    'BATHS',
    'BALCONY',
    'DESCRIPTION_LENGTH',
    'CITY',
    'LOCALITY',
    'PRICE_CATEGORY',
    'BATH_CATEGORY'
]

print("\nFeature columns set:")
print(feature_cols)



Loaded GOLD dataset:

Feature columns set:
['TOTAL_AREA', 'PRICE_PER_SQFT', 'BATHS', 'BALCONY', 'DESCRIPTION_LENGTH', 'CITY', 'LOCALITY', 'PRICE_CATEGORY', 'BATH_CATEGORY']


In [ ]:
# Cell 16-DIAG — MUST RUN BEFORE FIXING
print("Columns in df_gold:\n", df_gold.columns.tolist())

print("\nFeature cols you set:\n", feature_cols)

# Check which features are missing
missing = [c for c in feature_cols if c not in df_gold.columns]
print("\nMissing columns:", missing)


Columns in df_gold:
 ['PROPERTY_ID', 'NAME', 'PROPERTY_TITLE', 'PRICE', 'TOTAL_AREA', 'PRICE_PER_SQFT', 'BATHS', 'BALCONY', 'LOCATION', 'CITY', 'LOCALITY', 'DESCRIPTION_LENGTH', 'PRICE_CATEGORY', 'BATH_CATEGORY', 'DESCRIPTION']

Feature cols you set:
 ['TOTAL_AREA', 'PRICE_PER_SQFT', 'BATHS', 'BALCONY', 'DESCRIPTION_LENGTH', 'CITY', 'LOCALITY', 'PRICE_CATEGORY', 'BATH_CATEGORY']

Missing columns: []


In [ ]:
# FINAL Cell 16 — WITH BALCONY_FLAG FIX

import joblib
import pandas as pd
import os

print("Reloading GOLD dataset...")
df_gold = pd.read_csv("/content/RESIDENTIAL_GOLD.csv")   # <-- your exact file

print("Fixing BALCONY_FLAG...")
df_gold['BALCONY_FLAG'] = df_gold['BALCONY'].astype(int)

print("Loading best model...")
best_model = joblib.load("ai/model_best.pkl")

print("Generating predictions...")

# Raw columns + required BALCONY_FLAG
raw_input_cols = [
    'TOTAL_AREA',
    'PRICE_PER_SQFT',
    'BATHS',
    'BALCONY',
    'BALCONY_FLAG',        # <= THE FIX
    'DESCRIPTION_LENGTH',
    'CITY',
    'LOCALITY',
    'PRICE_CATEGORY',
    'BATH_CATEGORY'
]

X_full = df_gold[raw_input_cols]

# Predict
df_gold['PREDICTED_PRICE'] = best_model.predict(X_full)

# Create final output
df_pred_output = df_gold[['PROPERTY_ID','LOCATION','TOTAL_AREA','PRICE','PREDICTED_PRICE']].copy()
df_pred_output['PRICE'] = df_pred_output['PRICE'].round(2)
df_pred_output['PREDICTED_PRICE'] = df_pred_output['PREDICTED_PRICE'].round(2)

# Save file
os.makedirs("ai", exist_ok=True)
out_path = "ai/predicted_prices.csv"
df_pred_output.to_csv(out_path, index=False)

print("\nSaved predictions to:", out_path)
df_pred_output.head(10)



Reloading GOLD dataset...
Fixing BALCONY_FLAG...
Loading best model...
Generating predictions...

Saved predictions to: ai/predicted_prices.csv


,PROPERTY_ID,LOCATION,TOTAL_AREA,PRICE,PREDICTED_PRICE
0,0,"Kanathur Reddikuppam, Chennai",2583,19900000,19819000.0
1,1,"Ramanathan Nagar, Pozhichalur,Chennai",7000,22500000,23601000.0
2,2,"Kasthuribai Nagar, West Tambaram,Chennai",1320,10000000,9950000.0
3,3,"Naveenilaya,Chepauk, Triplicane,Chennai",4250,33300000,33221000.0
4,4,"Avadi, Chennai",960,4800000,4788000.0
5,5,"Siruseri, Chennai",940,4000000,4014000.0
6,6,"Thiran Flats ,Gowrivakkam, Sembakkam,Chennai",880,6000000,6006000.0
7,7,"Mahindra World City, Chennai",1700,7235000,7219650.0
8,8,"Brindavan Colony, West Tambaram,Chennai",840,4200000,4160640.0
9,9,"New Colony, Chromepet,Chennai",535,3000000,3006210.0


In [ ]:
!zip -r ai_files.zip ai/


  adding: ai/ (stored 0%)
  adding: ai/model_best.pkl (deflated 81%)
  adding: ai/predicted_prices.csv (deflated 74%)
